In [31]:
from dotenv import load_dotenv
from agents import Agent, Runner , trace , function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os 
from sendgrid.helpers.mail import Mail , Email ,To , Content

In [32]:
load_dotenv(override=True)

True

In [33]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [34]:
sales_agent1 = Agent("ciddi_eylemsel",instructions=instructions1,model="gpt-4o-mini")

sales_agent2 = Agent(
        name="Etkileyici Satış Temsilcisi",  # Engaging Sales Agent
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Meşgul Satış Temsilcisi",  # Busy Sales Agent
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [35]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit preparation through the power of AI.

Navigating the complexities of compliance can be time-consuming and resource-intensive. Our platform simplifies the process by automating key tasks, ensuring you meet all necessary requirements efficiently and effectively. By using ComplAI, companies like yours have significantly reduced their audit preparation time, allowing them to focus more on their core business objectives.

I would love to schedule a brief call to discuss how ComplAI can support your organization in achieving and maintaining SOC 2 compliance. Are you available for a 20-minute chat next week?

Looking forward to your response.

Best regards,

[Your Name]  
[Your Position]  
ComplAI  
[Your Phone Number]  
[Your Email Address]  
[Company Websi

In [36]:
import asyncio
from agents import Runner, trace
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplifying Your SOC 2 Compliance Journey

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with ComplAI, where we specialize in transforming the way businesses manage SOC 2 compliance and audit preparations.

In today's regulatory landscape, ensuring compliance can be both time-consuming and complex. Our AI-powered SaaS tool streamlines the process, enabling your team to focus on what they do best while we handle the intricacies of compliance management. 

Key benefits of our solution include:

- **Automated Documentation**: Rapidly generate and maintain compliance documentation, reducing manual effort and risk of errors.
- **Real-time Monitoring**: Stay updated with continuous compliance checks and alerts, ensuring you're always audit-ready.
- **Seamless Collaboration**: Enable cross-functional teams to work together effortlessly, promoting transparency and accountability.

I would love the opportunity to discuss how ComplAI can 

In [37]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [38]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Let’s Make Compliance as Easy as Pie (Without the Crust)!

Hi [Recipient's Name],

I hope this email finds you sipping coffee rather than drowning in a sea of compliance paperwork! Speaking of which, have you heard about ComplAI? We’re like your favorite coffee shop, but instead of brews, we serve up hassle-free SOC2 compliance and audit prep—all served hot and fresh with a side of AI!

Imagine a world where audits don’t make you break out in a cold sweat. With our tool, you can:
- **Automate** those pesky compliance tasks (goodbye, spreadsheets!)
- Ensure you stay on top of audit requirements without wearing a detective's hat
- Get back to the things that really matter—like perfecting your office playlist!

If you’re ready to trade in the compliance chaos for a more zen experience, let’s chat! No pressure—just a couple of minutes to explore how we can help you breathe easier around audit season (and maybe even clear some time for that coffee break).

Looking

In [43]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("cenghanbayram35@gmail.com")  # Change to your verified sender
    to_email = To("200316066@ogr.cbu.edu.tr")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [44]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000234C93C8A40>, strict_json_schema=True, is_enabled=True)

In [45]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000234C589FEC0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000234C8F3B4C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

In [46]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)